In [ ]:
!pip install geocoder
!pip install folium

In [ ]:
Collecting geocoder
  Downloading geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
     |████████████████████████████████| 98 kB 2.7 MB/s 
Requirement already satisfied: click in /srv/conda/envs/notebook/lib/python3.7/site-packages (from geocoder) (7.1.2)
Collecting ratelim
  Downloading ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
Requirement already satisfied: requests in /srv/conda/envs/notebook/lib/python3.7/site-packages (from geocoder) (2.25.1)
Requirement already satisfied: six in /srv/conda/envs/notebook/lib/python3.7/site-packages (from geocoder) (1.15.0)
Collecting future
  Downloading future-0.18.2.tar.gz (829 kB)
     |████████████████████████████████| 829 kB 8.3 MB/s 
Requirement already satisfied: decorator in /srv/conda/envs/notebook/lib/python3.7/site-packages (from ratelim->geocoder) (4.4.2)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->geocoder) (1.26.3)
Requirement already satisfied: chardet<5,>=3.0.2 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->geocoder) (4.0.0)
Requirement already satisfied: idna<3,>=2.5 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->geocoder) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->geocoder) (2020.12.5)
Building wheels for collected packages: future
  Building wheel for future (setup.py) ... done
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=35d90f0ba98726c6ab6ba2a1baf847576c9931288b24cedd82dd17dba34d3c79
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
Installing collected packages: ratelim, future, geocoder
Successfully installed future-0.18.2 geocoder-1.38.1 ratelim-0.1.6
jovyan@jupyter-jupyterlab-2djupyterlab-2ddemo-2dgiqbbmjb:~$ pip install folium
Collecting folium
  Downloading folium-0.12.1-py2.py3-none-any.whl (94 kB)
     |████████████████████████████████| 94 kB 2.6 MB/s 
Requirement already satisfied: requests in /srv/conda/envs/notebook/lib/python3.7/site-packages (from folium) (2.25.1)
Requirement already satisfied: numpy in /srv/conda/envs/notebook/lib/python3.7/site-packages (from folium) (1.20.2)
Requirement already satisfied: jinja2>=2.9 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from folium) (2.11.3)
Requirement already satisfied: branca>=0.3.0 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from folium) (0.4.2)
Requirement already satisfied: MarkupSafe>=0.23 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from jinja2>=2.9->folium) (1.1.1)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->folium) (1.26.3)
Requirement already satisfied: chardet<5,>=3.0.2 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->folium) (4.0.0)
Requirement already satisfied: certifi>=2017.4.17 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->folium) (2020.12.5)
Requirement already satisfied: idna<3,>=2.5 in /srv/conda/envs/notebook/lib/python3.7/site-packages (from requests->folium) (2.10)
Installing collected packages: folium
Successfully installed folium-0.12.1

In [14]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [8]:
# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [9]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned


In [10]:
df.describe()

,Postalcode,Borough,Neighborhood
count,20,20,20
unique,20,20,20
top,\nM1JScarborough(Scarborough Village)\n\n,\nM2ENot assigned\n\n,M9LNorth York(Humber Summit)
freq,1,1,1


In [11]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [12]:
df.head()

,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned


In [13]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [14]:
df_2.describe()

,Postalcode,Borough,Neighborhood
count,20,20,20
unique,20,20,20
top,\nM1JScarborough(Scarborough Village)\n\n,\nM2ENot assigned\n\n,M9LNorth York(Humber Summit)
freq,1,1,1


In [15]:
print(df_2.shape)
df_2.head()

(20, 3)


,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned


In [61]:
import geocoder
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [62]:
# Retrieving Postal Code Co-ordinates
postal_codes = df_2['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']


In [63]:
df_2[df_2.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [64]:
df_2.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue),43.648690,-79.385440
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,43.789480,-79.176140
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned,43.763430,-79.178200
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned,43.767480,-79.228290
5,\nM1HScarborough(Cedarbrae)\n\n,\nM2HNorth York(Hillcrest Village)\n\n,M9HNot assigned,43.741672,-79.232538
6,\nM1JScarborough(Scarborough Village)\n\n,\nM2JNorth York(Fairview / Henry Farm / Oriole...,M9JNot assigned,43.738650,-79.216990
7,\nM1KScarborough(Kennedy Park / Ionview / East...,\nM2KNorth York(Bayview Village)\n\n,M9KNot assigned,43.648690,-79.385440
8,\nM1LScarborough(Golden Mile / Clairlea / Oakr...,\nM2LNorth York(York Mills / Silver Hills)\n\n,M9LNorth York(Humber Summit),43.698830,-79.279120
9,\nM1MScarborough(Cliffside / Cliffcrest / Scar...,\nM2MNorth York(Willowdale / Newtonbrook)\n\n,M9MNorth York(Humberlea / Emery),43.726420,-79.232970


In [ ]:
address = 'Scarborough,Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Seattle,Washington are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Seattle,Washington are 43.773077, -79.257774.

In [ ]:
map_Scarborough = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

The Geograpical Co-ordinate of Neighborhood_1 are 43.773077, -79.257774.

In [78]:
#@hiddel_cell
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # my Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET: 1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


In [ ]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [ ]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

In [ ]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

In [ ]:
	name	categories	lat	lng
0	Disney Store	Toy / Game Store	43.775537	-79.256833
1	American Eagle Outfitters	Clothing Store	43.775908	-79.258352
2	SEPHORA	Cosmetics Shop	43.775017	-79.258109
3	Canyon Creek Chophouse	Steakhouse	43.776959	-79.261694
4	Coliseum Scarborough Cinemas	Movie Theater	43.775995	-79.255649

In [ ]:
# Top 10 Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

In [ ]:

def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# Nearby Venues
Scarborough_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )

In [ ]:
Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Bedford Park, Lawrence Manor East
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Lawrence Heights, Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
Downsview, North Park, Upwood Park
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Queen's Park
Canada Post Gateway Processing Centre
Business Reply Mail Processing Centre 969 Eastern
Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Humber Summit
Emery, Humberlea
Weston
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest

In [ ]:
print('There are {} Uniques Categories.'.format(len(Scarborough_venues['Venue Category'].unique())))
Scarborough_venues.groupby('Neighborhood').count().head()

There are 299 Uniques Categories.

In [ ]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [126]:
---- Adelaide, King, Richmond ----
          venue  freq
0   Coffee Shop  0.09
1          Café  0.07
2         Hotel  0.06
3  Burger Joint  0.03
4           Bar  0.03


---- Agincourt ----
                  venue  freq
0         Shopping Mall  0.07
1    Chinese Restaurant  0.07
2  Hong Kong Restaurant  0.03
3   Filipino Restaurant  0.03
4   Shanghai Restaurant  0.03


---- Agincourt North, L'Amoreaux East, Milliken, Steeles East ----
              venue  freq
0          Pharmacy  0.50
1    Sandwich Place  0.25
2  Sushi Restaurant  0.25
3       Zoo Exhibit  0.00
4     Movie Theater  0.00


---- Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown ----
                 venue  freq
0        Grocery Store  0.17
1       Hardware Store  0.08
2  Fried Chicken Joint  0.08
3       Sandwich Place  0.08
4       Discount Store  0.08


---- Alderwood, Long Branch ----
               venue  freq
0  Convenience Store  0.11
1        Coffee Shop  0.11
2     Sandwich Place  0.11
3       Dance Studio  0.11
4                Pub  0.11


---- Bathurst Manor, Downsview North, Wilson Heights ----
                       venue  freq
0                Pizza Place  0.18
1                Coffee Shop  0.09
2   Mediterranean Restaurant  0.09
3              Deli / Bodega  0.09
4  Middle Eastern Restaurant  0.09


---- Bayview Village ----
              venue  freq
0  Asian Restaurant  0.25
1       Flower Shop  0.25
2              Park  0.25
3             Trail  0.25
4       Zoo Exhibit  0.00


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0    Sushi Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.07
3                Café  0.04
4           Juice Bar  0.04


---- Berczy Park ----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.05
2         Café  0.05
3        Hotel  0.05
4     Beer Bar  0.04


---- Birch Cliff, Cliffside West ----
                   venue  freq
0                   Park  0.22
1         Baseball Field  0.11
2           Skating Rink  0.11
3  General Entertainment  0.11
4                    Gym  0.11


---- Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe ----
               venue  freq
0     Baseball Field  0.09
1  Fish & Chips Shop  0.09
2     Sandwich Place  0.09
3      Shopping Mall  0.09
4      Grocery Store  0.09


---- Brockton, Exhibition Place, Parkdale Village ----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.07
2                     Bar  0.06
3              Restaurant  0.04
4  Furniture / Home Store  0.04


---- Business Reply Mail Processing Centre 969 Eastern ----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.06
2             Hotel  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


---- CFB Toronto, Downsview East ----
          venue  freq
0  Soccer Field   0.2
1    Food Court   0.2
2       Airport   0.2
3          Park   0.2
4   Coffee Shop   0.2


---- CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara ----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.05
2                Café  0.05
3  Italian Restaurant  0.05
4                 Gym  0.04


---- Cabbagetown, St. James Town ----
                 venue  freq
0           Restaurant  0.06
1          Coffee Shop  0.06
2          Pizza Place  0.05
3  Japanese Restaurant  0.03
4               Bakery  0.03


---- Caledonia-Fairbanks ----
                 venue  freq
0                 Park  0.17
1  Sporting Goods Shop  0.08
2             Bus Stop  0.08
3                 Café  0.08
4               Market  0.08


---- Canada Post Gateway Processing Centre ----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.06
2             Hotel  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


---- Cedarbrae ----
                  venue  freq
0  Caribbean Restaurant  0.12
1           Flower Shop  0.12
2     Indian Restaurant  0.12
3                  Bank  0.12
4                Bakery  0.12


---- Central Bay Street ----
                venue  freq
0         Coffee Shop  0.06
1      Clothing Store  0.04
2      Cosmetics Shop  0.04
3      Ice Cream Shop  0.03
4  Italian Restaurant  0.03


---- Chinatown, Grange Park, Kensington Market ----
                           venue  freq
0                           Café  0.06
1             Chinese Restaurant  0.05
2                            Bar  0.05
3          Vietnamese Restaurant  0.04
4  Vegetarian / Vegan Restaurant  0.04


---- Christie ----
               venue  freq
0  Korean Restaurant  0.18
1      Grocery Store  0.10
2              Diner  0.04
3  Indian Restaurant  0.04
4               Café  0.04


---- Church and Wellesley ----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.04
2                 Café  0.03
3     Sushi Restaurant  0.03
4           Restaurant  0.03


---- Clairlea, Golden Mile, Oakridge ----
          venue  freq
0  Intersection  0.17
1      Bus Line  0.11
2   Coffee Shop  0.11
3         Diner  0.11
4        Bakery  0.11


---- Clarks Corners, Sullivan, Tam O'Shanter ----
                   venue  freq
0   Fast Food Restaurant  0.10
1            Pizza Place  0.10
2  Vietnamese Restaurant  0.07
3            Golf Course  0.03
4                 Market  0.03


---- Cliffcrest, Cliffside, Scarborough Village West ----
                  venue  freq
0  Fast Food Restaurant  0.17
1                  Park  0.08
2           Flower Shop  0.08
3          Liquor Store  0.08
4           Pizza Place  0.08


---- Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park ----
                venue  freq
0         Pizza Place  0.25
1   Convenience Store  0.12
2  Mexican Restaurant  0.12
3                Bank  0.12
4                 Gym  0.12


---- Commerce Court, Victoria Hotel ----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.06
2         Café  0.05
3   Restaurant  0.04
4    Gastropub  0.04


---- Davisville ----
            venue  freq
0    Dessert Shop  0.10
1     Pizza Place  0.10
2     Coffee Shop  0.07
3  Sandwich Place  0.07
4             Gym  0.05


---- Davisville North ----
            venue  freq
0             Gym  0.09
1     Yoga Studio  0.04
2        Pharmacy  0.04
3  Breakfast Spot  0.04
4         Brewery  0.04


---- Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West ----
              venue  freq
0               Pub  0.10
1       Coffee Shop  0.10
2      Skating Rink  0.10
3  Sushi Restaurant  0.10
4       Yoga Studio  0.05


---- Del Ray, Keelesdale, Mount Dennis, Silverthorn ----
                        venue  freq
0  Construction & Landscaping   0.2
1                  Playground   0.2
2            Storage Facility   0.2
3                 Coffee Shop   0.2
4        Fast Food Restaurant   0.2


---- Design Exchange, Toronto Dominion Centre ----
         venue  freq
0  Coffee Shop  0.13
1         Café  0.08
2        Hotel  0.07
3   Restaurant  0.04
4    Gastropub  0.03


---- Don Mills North ----
                venue  freq
0         Coffee Shop  0.11
1        Burger Joint  0.05
2  Italian Restaurant  0.05
3        Soccer Field  0.05
4       Smoothie Shop  0.05


---- Dorset Park, Scarborough Town Centre, Wexford Heights ----
         venue  freq
0       Bakery  0.33
1         Park  0.33
2    Wine Shop  0.17
3  Coffee Shop  0.17
4  Zoo Exhibit  0.00


---- Dovercourt Village, Dufferin ----
          venue  freq
0           Bar  0.10
1   Coffee Shop  0.10
2        Bakery  0.08
3          Café  0.06
4  Cocktail Bar  0.04


---- Downsview Central ----
               venue  freq
0       Home Service  0.29
1      Moving Target  0.14
2  Mobile Phone Shop  0.14
3               Park  0.14
4     Baseball Field  0.14


---- Downsview Northwest ----
                   venue  freq
0          Grocery Store  0.07
1            Pizza Place  0.07
2   Fast Food Restaurant  0.07
3  Vietnamese Restaurant  0.07
4          Shopping Mall  0.07


---- Downsview West ----
            venue  freq
0     Coffee Shop  0.10
1   Grocery Store  0.05
2  Discount Store  0.05
3  Sandwich Place  0.05
4   Big Box Store  0.05


---- Downsview, North Park, Upwood Park ----
                        venue  freq
0           Convenience Store   0.2
1  Construction & Landscaping   0.2
2                Home Service   0.2
3                      Bakery   0.2
4                        Park   0.2


---- East Birchmount Park, Ionview, Kennedy Park ----
               venue  freq
0  Convenience Store  0.25
1     Discount Store  0.25
2        Coffee Shop  0.25
3   Department Store  0.25
4        Music Venue  0.00


---- East Toronto ----
                venue  freq
0      Sandwich Place  0.14
1  Italian Restaurant  0.14
2         Coffee Shop  0.14
3                Park  0.14
4  Athletics & Sports  0.14


---- Emery, Humberlea ----
            venue  freq
0     Coffee Shop  0.29
1            Park  0.14
2  Discount Store  0.14
3            Café  0.14
4       Nightclub  0.14


---- Fairview, Henry Farm, Oriole ----
                  venue  freq
0        Clothing Store  0.16
1  Fast Food Restaurant  0.10
2           Coffee Shop  0.07
3         Women's Store  0.04
4                Bakery  0.03


---- First Canadian Place, Underground city ----
                 venue  freq
0          Coffee Shop  0.14
1                Hotel  0.08
2                 Café  0.07
3            Gastropub  0.03
4  American Restaurant  0.03


---- Flemingdon Park, Don Mills South ----
                 venue  freq
0  Japanese Restaurant  0.08
1       Science Museum  0.08
2     Asian Restaurant  0.08
3           Beer Store  0.08
4                  Gym  0.08


---- Forest Hill North, Forest Hill West ----
            venue  freq
0            Café   0.2
1     Coffee Shop   0.1
2       Bookstore   0.1
3    Burger Joint   0.1
4  Ice Cream Shop   0.1


---- Glencairn ----
                      venue  freq
0      Fast Food Restaurant  0.18
1             Grocery Store  0.18
2             Metro Station  0.09
3       Japanese Restaurant  0.09
4  Mediterranean Restaurant  0.09


---- Guildwood, Morningside, West Hill ----
                  venue  freq
0                  Park  0.29
1                  Pool  0.14
2     Indian Restaurant  0.14
3   Fried Chicken Joint  0.14
4  Gym / Fitness Center  0.14


---- Harbord, University of Toronto ----
         venue  freq
0         Café  0.07
1   Restaurant  0.06
2  Coffee Shop  0.05
3  Pizza Place  0.04
4          Bar  0.04


---- Harbourfront East, Toronto Islands, Union Station ----
             venue  freq
0             Café  0.29
1  Harbor / Marina  0.14
2             Park  0.14
3             Pier  0.14
4            Beach  0.14


---- Harbourfront, Regent Park ----
         venue  freq
0  Coffee Shop  0.12
1      Theater  0.06
2       Bakery  0.06
3         Café  0.04
4   Restaurant  0.04


---- High Park, The Junction South ----
               venue  freq
0  Convenience Store  0.09
1               Park  0.07
2               Café  0.07
3                Bar  0.07
4    Thai Restaurant  0.04


---- Highland Creek, Rouge Hill, Port Union ----
          venue  freq
0           Bar   1.0
1        Museum   0.0
2   Opera House   0.0
3        Office   0.0
4  Noodle House   0.0


---- Hillcrest Village ----
                                      venue  freq
0                                      Park  0.29
1                      Fast Food Restaurant  0.14
2                                    Bakery  0.14
3  Residential Building (Apartment / Condo)  0.14
4                                  Pharmacy  0.14


---- Humber Bay Shores, Mimico South, New Toronto ----
               venue  freq
0               Park  0.14
1      Grocery Store  0.10
2  Convenience Store  0.10
3       Skating Rink  0.05
4     Breakfast Spot  0.05


---- Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea ----
                venue  freq
0         Coffee Shop  0.19
1      Sandwich Place  0.06
2  Italian Restaurant  0.06
3        Liquor Store  0.06
4                Bank  0.06


---- Humber Summit ----
               venue  freq
0  Electronics Store   1.0
1        Zoo Exhibit   0.0
2        Opera House   0.0
3       Noodle House   0.0
4          Nightclub   0.0


---- Humewood-Cedarvale ----
               venue  freq
0  Convenience Store  0.14
1              Trail  0.14
2       Hockey Arena  0.14
3      Grocery Store  0.14
4      Deli / Bodega  0.14


---- Islington Avenue ----
               venue  freq
0           Pharmacy  0.25
1  Convenience Store  0.08
2      Shopping Mall  0.08
3      Grocery Store  0.08
4               Bank  0.08


---- Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips ----
                 venue  freq
0             Pharmacy   0.2
1    Mobile Phone Shop   0.2
2             Bus Line   0.2
3  American Restaurant   0.2
4          Coffee Shop   0.2


---- Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor ----
             venue  freq
0    Burrito Place  0.12
1              Gym  0.08
2             Bank  0.04
3    Grocery Store  0.04
4  Thai Restaurant  0.04


---- L'Amoreaux West ----
                  venue  freq
0              Pharmacy  0.13
1  Fast Food Restaurant  0.13
2    Chinese Restaurant  0.13
3  Other Great Outdoors  0.07
4         Grocery Store  0.07


---- Lawrence Heights, Lawrence Manor ----
              venue  freq
0    Clothing Store  0.17
1      Dessert Shop  0.05
2  Toy / Game Store  0.03
3        Restaurant  0.03
4  Greek Restaurant  0.03


---- Lawrence Park ----
         venue  freq
0     Bus Line  0.14
1         Park  0.14
2         Café  0.14
3  Coffee Shop  0.14
4   Restaurant  0.14


---- Leaside ----
                    venue  freq
0             Coffee Shop  0.10
1       Electronics Store  0.06
2     Sporting Goods Shop  0.06
3  Furniture / Home Store  0.06
4                 Brewery  0.04


---- Little Portugal, Trinity ----
                venue  freq
0                 Bar  0.08
1         Coffee Shop  0.05
2          Restaurant  0.05
3                Café  0.04
4  Italian Restaurant  0.04


---- Maryvale, Wexford ----
                       venue  freq
0                 Hookah Bar   0.2
1                Pizza Place   0.2
2  Middle Eastern Restaurant   0.2
3                Gas Station   0.2
4               Burger Joint   0.2


---- Moore Park, Summerhill East ----
             venue  freq
0             Park   0.4
1    Grocery Store   0.2
2              Gym   0.2
3  Thai Restaurant   0.2
4      Zoo Exhibit   0.0


---- Newtonbrook, Willowdale ----
                       venue  freq
0          Korean Restaurant  0.12
1  Middle Eastern Restaurant  0.12
2                       Café  0.09
3                Coffee Shop  0.09
4        Fried Chicken Joint  0.03


---- North Toronto West ----
                 venue  freq
0          Coffee Shop  0.09
1  Sporting Goods Shop  0.09
2                 Café  0.09
3       Clothing Store  0.09
4           Restaurant  0.06


---- Northwest ----
              venue  freq
0    Sandwich Place  0.17
1       Coffee Shop  0.17
2  Storage Facility  0.17
3      Burger Joint  0.17
4               Gym  0.17


---- Northwood Park, York University ----
                 venue  freq
0  Japanese Restaurant  0.12
1                  Bar  0.12
2                 Bank  0.12
3          Pizza Place  0.12
4       Massage Studio  0.12


---- Parkdale, Roncesvalles ----
                         venue  freq
0                  Coffee Shop  0.09
1                       Bakery  0.06
2  Eastern European Restaurant  0.04
3             Sushi Restaurant  0.04
4              Thai Restaurant  0.04


---- Parkwoods ----
                       venue  freq
0          Food & Drink Shop  0.33
1                       Park  0.33
2                  Pet Store  0.33
3  Middle Eastern Restaurant  0.00
4         Miscellaneous Shop  0.00


---- Queen's Park ----
                 venue  freq
0          Coffee Shop  0.20
1                 Café  0.08
2       Sandwich Place  0.05
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.04


---- Rosedale ----
           venue  freq
0           Park   0.4
1     Playground   0.2
2  Grocery Store   0.2
3    Candy Store   0.2
4    Zoo Exhibit   0.0


---- Roselawn ----
              venue  freq
0        Playground   0.4
1            Garden   0.2
2  Business Service   0.2
3         Pet Store   0.2
4       Zoo Exhibit   0.0


---- Rouge, Malvern ----
                        venue  freq
0                 Zoo Exhibit  0.25
1  Financial or Legal Service  0.25
2        Fast Food Restaurant  0.25
3  Construction & Landscaping  0.25
4           Hotpot Restaurant  0.00


---- Runnymede, Swansea ----
              venue  freq
0              Café  0.10
1       Coffee Shop  0.08
2            Bakery  0.05
3       Pizza Place  0.05
4  Sushi Restaurant  0.03


---- Ryerson, Garden District ----
                       venue  freq
0                Coffee Shop  0.07
1             Clothing Store  0.05
2             Cosmetics Shop  0.04
3  Middle Eastern Restaurant  0.04
4                 Restaurant  0.03


---- Scarborough Village ----
                  venue  freq
0  Fast Food Restaurant  0.33
1        Sandwich Place  0.22
2         Train Station  0.11
3         Big Box Store  0.11
4     Indian Restaurant  0.11


---- Silver Hills, York Mills ----
           venue  freq
0           Park   1.0
1    Zoo Exhibit   0.0
2  Moving Target   0.0
3   Noodle House   0.0
4      Nightclub   0.0


---- St. James Town ----
                venue  freq
0         Coffee Shop  0.07
1               Hotel  0.06
2                Café  0.06
3  Italian Restaurant  0.04
4              Bakery  0.04


---- Stn A PO Boxes 25 The Esplanade ----
              venue  freq
0       Coffee Shop  0.10
1              Café  0.06
2             Hotel  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


---- Studio District ----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2  Italian Restaurant  0.05
3               Diner  0.05
4              Bakery  0.05


---- The Annex, North Midtown, Yorkville ----
                venue  freq
0  Italian Restaurant  0.08
1                Café  0.08
2         Coffee Shop  0.08
3      Sandwich Place  0.08
4      History Museum  0.05


---- The Beaches ----
                 venue  freq
0                  Pub  0.06
1          Pizza Place  0.06
2  Japanese Restaurant  0.06
3                  Bar  0.06
4                 Café  0.03


---- The Beaches West, India Bazaar ----
                venue  freq
0                Park  0.08
1                Café  0.05
2              Bakery  0.05
3  Italian Restaurant  0.05
4             Brewery  0.05


---- The Danforth West, Riverdale ----
              venue  freq
0  Greek Restaurant  0.13
1    Ice Cream Shop  0.07
2              Café  0.07
3       Yoga Studio  0.03
4   Bubble Tea Shop  0.03


---- The Junction North, Runnymede ----
                    venue  freq
0                 Brewery  0.38
1                    Park  0.25
2      Athletics & Sports  0.12
3             Coffee Shop  0.12
4  Furniture / Home Store  0.12


---- The Kingsway, Montgomery Road, Old Mill North ----
               venue  freq
0   Sushi Restaurant  0.07
1     Breakfast Spot  0.07
2  French Restaurant  0.07
3       Burger Joint  0.07
4   Toy / Game Store  0.04


---- Thorncliffe Park ----
               venue  freq
0       Burger Joint  0.11
1  Indian Restaurant  0.11
2        Yoga Studio  0.06
3     Discount Store  0.06
4             Bridge  0.06


---- Victoria Village ----
                       venue  freq
0  Middle Eastern Restaurant  0.14
1      Portuguese Restaurant  0.14
2                Pizza Place  0.14
3          French Restaurant  0.14
4                       Park  0.14


---- Westmount ----
                venue  freq
0         Pizza Place  0.18
1  Golf Driving Range  0.09
2      Discount Store  0.09
3         Supermarket  0.09
4  Chinese Restaurant  0.09


---- Weston ----
                       venue  freq
0              Train Station  0.15
1                Coffee Shop  0.15
2             Sandwich Place  0.08
3  Middle Eastern Restaurant  0.08
4                Pizza Place  0.08


---- Willowdale South ----
              venue  freq
0       Coffee Shop  0.07
1  Ramen Restaurant  0.05
2       Pizza Place  0.05
3              Café  0.05
4    Sandwich Place  0.05


---- Willowdale West ----
                         venue  freq
0                  Coffee Shop  0.15
1                         Park  0.08
2  Eastern European Restaurant  0.08
3                Grocery Store  0.08
4                       Bakery  0.08


---- Woburn ----
                  venue  freq
0           Coffee Shop   0.4
1                  Park   0.2
2      Business Service   0.2
3  Fast Food Restaurant   0.2
4           Zoo Exhibit   0.0


---- Woodbine Gardens, Parkview Hill ----
                  venue  freq
0  Fast Food Restaurant  0.10
1               Brewery  0.10
2           Pizza Place  0.10
3          Home Service  0.10
4            Restaurant  0.05


---- Woodbine Heights ----
            venue  freq
0        Bus Line  0.06
1     Pizza Place  0.06
2     Coffee Shop  0.06
3   Grocery Store  0.06
4  Sandwich Place  0.06


---- York Mills West ----
           venue  freq
0    Coffee Shop  0.16
1     Restaurant  0.12
2            Gym  0.12
3  Burrito Place  0.04
4           Bank  0.04


SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# Using K-Means to cluster neighborhood into 3 clusters
Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(Scarborough_grouped_clustering)
kmeans.labels_
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged =df_2.iloc[:16,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head()# check the last columns!

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_merged =df_2.iloc[:16,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head()# check the last columns!

In [ ]:

kclusters = 10

In [ ]:
 create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Scarborough_merged['Latitude'], 
                                   Scarborough_merged['Longitude'], 
                                   Scarborough_merged['Neighborhood'], 
                                   Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']

In [ ]:
df1=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]
df2=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]
df3=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

Scarborough_Avg_HousingPrice=pd.DataFrame({"Neighborhood":df_2["Neighborhood"],
                                      "Average_Housing_Price":[335000.0,286600.0,175000.0,225900.0,219400.0,
                                                               573900.0,225000.0,370500.0,370500.0,433500.0,279200.0,
                                                               279200.0,225000.0,370500.,255400.0,433500.0,433500.0,
                                                               435000.0,289500.0,265000.0,285900.0,239400.0,
                                                               589900.0,295000.0,380500.0,378500.0,438500.0,229200.0,
                                                               229200.0,365000.0,388500.,285400.0,493500.0,477500.0,378000.0,316600.0,195000.0,225900.0,219400.0,
                                                               573900.0,367000.0,370500.0,370500.0,363500.0,279200.0,
                                                               279200.0,271000.0,370500.,255400.0,383500.0,433500.0,335000.0,286600.0,185000.0,225900.0,219400.0,
                                                               573900.0,329000.0,370500.0,370500.0,533500.0,279200.0,
                                                               279200.0,375000.0,370500.,255400.0,493500.0,433500.0,335000.0,286600.0,165000.0,225900.0,219400.0,
                                                               573900.0,425000.0,370500.0,370500.0,433500.0,279200.0,
                                                               279200.0,195000.0,370500.,255400.0,403500.0,433500.0,335000.0,286600.0,187000.0,225900.0,219400.0,
                                                               573900.0,325000.0,370500.0,370500.0,333500.0,279200.0,
                                                               279200.0,289000.0,370500.,255400.0,413500.0,433500.0,254800.0
                                                              
                                                              
                                                              ]})

Scarborough_Avg_HousingPrice.set_index('Neighborhood',inplace=True,drop=True)

Scarborough_Avg_HousingPrice.plot(kind='bar',figsize=(24,18),alpha=0.75)



In [125]:
import matplotlib.pyplot as plt
%matplotlib inline
clusters=pd.DataFrame({"Cluster1":df1["Neighborhood"],
                      "Cluster2":df2["Neighborhood"],
                      "Cluster4":df3["Neighborhood"]})
clusters = clusters.replace(np.nan, '', regex=True)

new_Scarborough=Scarborough_merged.set_index("Neighborhood",drop=True)
#Source:https://www.greatschools.org
Scarborough_school_ratings=pd.DataFrame({"Neighborhood":df["Neighborhood"],
                                      "Top School Rating":[7,9,5,8,10,10,7,10,1,2,1,2,7,2,3,2,6,
                                                           5,4,8,9,9,6,6,4,5,4,6,8,10,8,9,6,2,
                                                           10,2,5,8,9,6,6,10,8,9,1,2,3,4,5,6,9,
                                                           8,5,9,6,9,6,4,8,10,2,5,6,3,9,8,7,
                                                           7,8,5,8,9,1,5,4,7,2,3,6,6,9,4,8,7,
                                                           4,8,9,2,6,4,7,5,10,4,6,8,9,7,5,6,5,8,7
                                                           ]
                                         
Scarborough_school_ratings.set_index('Neighborhood',inplace=True,drop=True)

Scarborough_school_ratings.plot(kind='bar',figsize=(16,10),color='green',alpha=0.75);

SyntaxError: invalid syntax (<string>, line 19)